<center><img src="images/DLI_Header.png" alt="标题" style="width: 400px;"/></center>

# 5. CLIP

对比语言-图像预训练或 [CLIP](https://github.com/openai/CLIP/tree/main) 是一种文本和图像编码工具，可与许多流行的生成式 AI 模型（例如 [DALL-E](https://openai.com/dall-e-2) 和 [Stable Diffusion](https://github.com/Stability-AI/stablediffusion)）一起使用。

CLIP 本身并不是生成式 AI 模型，而是用于将文本编码与图像编码对齐。如果存在完美的图像文本描述，那么 CLIP 的目标就是为图像和文本创建相同的向量嵌入。让我们看看这在实践中意味着什么。

本笔记本的目标是：
* 学习如何使用 CLIP 编码
  * 获取图像编码
  * 获取文本编码
  * 计算它们之间的余弦相似度
* 使用 CLIP 创建文本到图像的神经网络

## 5.1 编码

首先，让我们加载本练习所需的库。

In [ ]:
import csv
import glob
import numpy as np
import torch
import torch.nn.functional as F
from torch.optim import Adam
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import clip

# Visualization tools
import matplotlib.pyplot as plt
from PIL import Image
from torchvision.utils import save_image, make_grid
from textwrap import wrap

# User defined libraries
from utils import other_utils
from utils import ddpm_utils
from utils import UNet_utils

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

基于流行的图像识别神经网络，CLIP 有几种不同的变体：
* RN50
* RN101
* RN50x4
* RN50x16
* RN50x64
* ViT-B/32
* ViT-B/16
* ViT-L/14
* ViT-L/14@336px

In [ ]:
# clip.available_models()

对于此笔记本，我们将使用基于 [Vision Transformer](https://huggingface.co/docs/transformers/main/model_doc/vit) 架构的 `ViT-B/32` 。它具有 `512` 个特征，我们稍后会将其输入到我们的扩散模型中。

In [ ]:
clip_model, clip_preprocess = clip.load("ViT-B/32")
clip_model.eval()
CLIP_FEATURES = 512

### 5.1.1 图像编码

当我们加载 CLIP 时，它还会附带一组图像转换，我们可以使用这些图像将图像输入 CLIP 模型：

In [ ]:
clip_preprocess

我们可以在一张花卉照片上测试一下。首先从一朵雏菊开始。

In [ ]:
DATA_DIR = "data/cropped_flowers/"
img_path = DATA_DIR + "daisy/2877860110_a842f8b14a_m.jpg"
img = Image.open(img_path)
img.show()

我们可以先使用 `clip_preprocess` 转换图像，然后将结果转换为张量，从而找到 CLIP 嵌入。由于 `clip_model` 需要一批图像，因此我们可以使用 [np.stack](https://numpy.org/doc/stable/reference/generated/numpy.stack.html) 将处理后的图像转换为单个元素批次。

In [ ]:
clip_imgs = torch.tensor(np.stack([clip_preprocess(img)])).to(device)
clip_imgs.size()

然后，我们可以将批处理传递给 `clip_model.encode_image` 以查找图像的嵌入。如果您想查看编码的样子，请取消注释 `clip_img_encoding` 。当我们打印尺寸时，它会为我们的 `1` 图像列出 `512` 个特征。

In [ ]:
clip_img_encoding = clip_model.encode_image(clip_imgs)
print(clip_img_encoding.size())
#clip_img_encoding

### 5.1.2 文本编码

现在我们有了图像编码，让我们看看是否可以得到匹配的文本编码。下面是不同花卉描述的列表。与图像一样，文本需要经过预处理才能由 CLIP 编码。为此，CLIP 附带了一个 `tokenize` 函数，以便将每个单词转换为整数。

In [ ]:
text_list = [
    "A white daisy with a yellow center",
    "An orange sunflower with a big brown center",
    "A red rose bud"
]
text_tokens = clip.tokenize(text_list).to(device)
text_tokens

然后，我们可以将标记传递给 `encode_text` 以获取我们的文本编码。如果您想查看编码的样子，请取消注释 `clip_text_encodings` 。与我们的图像编码类似，我们的 `3` 幅图像中的每一幅都有 `512` 个特征。

In [ ]:
clip_text_encodings = clip_model.encode_text(text_tokens).float()
print(clip_text_encodings.size())
#clip_text_encodings

### 5.1.3 相似度

为了了解我们的哪一个文本描述最能描述雏菊，我们可以计算文本编码和图像编码之间的[余弦相似度](https://medium.com/@milana.shxanukova15/cosine-distance-and-cosine-similarity-a5da0e4d9ded)。当余弦相似度为 `1` 时，它们是完美匹配的。当余弦相似度为 `-1` 时，这两个编码是相反的。

余弦相似度相当于[点积](https://mathworld.wolfram.com/DotProduct.html)，每个向量都按其幅度归一化。换句话说，每个向量的幅度变为 `1` 。

我们可以使用以下公式来计算点积：

$X \cdot Y = \sum_{i=1}^{n} x_i y_i = x_1y_1 + x_2 y_2 + \cdots + x_n y_n$

In [ ]:
clip_img_encoding /= clip_img_encoding.norm(dim=-1, keepdim=True)
clip_text_encodings /= clip_text_encodings.norm(dim=-1, keepdim=True)
similarity = (clip_text_encodings * clip_img_encoding).sum(-1)
similarity

觉得如何？描述性最强的文字能获得最高分吗？

In [ ]:
for idx, text in enumerate(text_list):
    print(text, " - ", similarity[idx])

让我们再练习一下。下面，我们添加了一朵向日葵和一朵玫瑰的图片。

In [ ]:
img_paths = [
    DATA_DIR + "daisy/2877860110_a842f8b14a_m.jpg",
    DATA_DIR + "sunflowers/2721638730_34a9b7a78b.jpg",
    DATA_DIR + "roses/8032328803_30afac8b07_m.jpg"
]

imgs = [Image.open(path) for path in img_paths]
for img in imgs:
    img.show()

**TODO**：下面的 `get_img_encodings` 函数充斥着 `FIXMEs` 。请用适当的代码替换每个 `FIXME` ，以便从 PIL 图像生成 CLIP 编码。

单击 `...` 获取答案。

In [ ]:
def get_img_encodings(imgs):
    processed_imgs = [FIXME(img) for img in imgs]
    clip_imgs = torch.tensor(np.stack(FIXME)).to(device)
    clip_img_encodings = FIXME.encode_image(clip_imgs)
    return clip_img_encodings

In [ ]:
def get_img_encodings(imgs):
    processed_imgs = [clip_preprocess(img) for img in imgs]
    clip_imgs = torch.tensor(np.stack(processed_imgs)).to(device)
    clip_img_encodings = clip_model.encode_image(clip_imgs)
    return clip_img_encodings

In [ ]:
clip_img_encodings = get_img_encodings(imgs)
clip_img_encodings

**TODO**：找到能够很好地描述上述图像并产生高相似度分数的文本。计算相似度分数后，请随意重复此练习并进行修改。我们稍后会再次使用此文本列表。

单击 `...` 查看示例。

In [ ]:
text_list = [
    "A daisy",
    "A sunflower",
    "A rose"
]

```python
text_list = [
    "A white daisy with a yellow center",
    "An orange sunflower with a big brown center",
    "A deep red rose flower"
]
```

In [ ]:
text_tokens = clip.tokenize(text_list).to(device)
clip_text_encodings = clip_model.encode_text(text_tokens).float()
clip_text_encodings

最好能比较文本和图像的每种组合。为此，我们可以对每种图像编码 [重复](https://pytorch.org/docs/stable/generated/torch.Tensor.repeat.html#torch.Tensor.repeat) 每种文本编码。同样，我们可以对每种文本编码 [重复交错](https://pytorch.org/docs/stable/generated/torch.repeat_interleave.html) 每种图像编码。

In [ ]:
clip_img_encodings /= clip_img_encodings.norm(dim=-1, keepdim=True)
clip_text_encodings /= clip_text_encodings.norm(dim=-1, keepdim=True)

n_imgs = len(imgs)
n_text = len(text_list)

In [ ]:
repeated_clip_text_encodings = clip_text_encodings.repeat(n_imgs, 1)
repeated_clip_text_encodings

In [ ]:
repeated_clip_img_encoding = clip_img_encodings.repeat_interleave(n_text, dim=0)
repeated_clip_img_encoding

In [ ]:
similarity = (repeated_clip_text_encodings * repeated_clip_img_encoding).sum(-1)
similarity = torch.unflatten(similarity, 0, (n_text, n_imgs))
similarity

让我们比较一下。理想情况下，从左上角到右下角的对角线应该是亮黄色，与它们的高值相对应。其余的值应该是低值和蓝色。

In [ ]:
fig = plt.figure(figsize=(10, 10))
gs = fig.add_gridspec(2, 3, wspace=.1, hspace=0)

for i, img in enumerate(imgs):
    ax = fig.add_subplot(gs[0, i])
    ax.axis("off")
    plt.imshow(img)

ax = fig.add_subplot(gs[1, :])
plt.imshow(similarity.detach().cpu().numpy().T, vmin=0.1, vmax=0.3)

labels = [ '\n'.join(wrap(text, 20)) for text in text_list ]
plt.yticks(range(n_text), labels, fontsize=10)
plt.xticks([])

for x in range(similarity.shape[1]):
    for y in range(similarity.shape[0]):
        plt.text(x, y, f"{similarity[x, y]:.2f}", ha="center", va="center", size=12)

## 5.2 CLIP 数据集

在之前的笔记本中，我们使用花卉类别作为标签。这次，我们将使用 CLIP 编码作为标签。

如果 CLIP 的目标是将文本编码与图像编码对齐，那么我们是否需要为数据集中的每个图像提供文本描述？假设：我们不需要文本描述，只需要图像 CLIP 编码来创建文本到图像的管道。

为了测试这一点，让我们将 CLIP 编码作为“标签”添加到我们的数据集中。在每一批数据增强图像上运行 CLIP 会更准确，但速度也会更慢。我们可以通过预处理和提前存储编码来加快速度。

我们可以使用 [glob](https://docs.python.org/3/library/glob.html) 列出我们所有的图像文件路径：

In [ ]:
data_paths = glob.glob(DATA_DIR + '*/*.jpg', recursive=True)
data_paths[:5]

下一个代码块针对每个文件路径运行以下循环：
* 打开与路径关联的图像并将其存储在 `img` 中
* 预处理图像，找到 CLIP 编码，并将其存储在 `clip_img` 中
* 将 CLIP 编码从张量转换为 python 列表
* 将文件路径和 CLIP 编码作为一行存储在 csv 文件中

In [ ]:
csv_path = 'clip.csv'

with open(csv_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    for idx, path in enumerate(data_paths):
        img = Image.open(path)
        clip_img = torch.tensor(np.stack([clip_preprocess(img)])).to(device)
        label = clip_model.encode_image(clip_img)[0].tolist()
        writer.writerow([path] + label)

处理完整数据集可能需要几秒钟。完成后，打开 [clip.csv](clip.csv) 查看结果。

我们可以使用与其他笔记本相同的图像转换：

In [ ]:
IMG_SIZE = 32 # Due to stride and pooling, must be divisible by 2 multiple times
IMG_CH = 3
BATCH_SIZE = 128
INPUT_SIZE = (IMG_CH, IMG_SIZE, IMG_SIZE)

pre_transforms = [
    transforms.Resize(IMG_SIZE),
    transforms.ToTensor(),  # Scales data into [0,1]
    transforms.Lambda(lambda t: (t * 2) - 1)  # Scale between [-1, 1]
]
pre_transforms = transforms.Compose(pre_transforms)
random_transforms = [
    transforms.RandomCrop(IMG_SIZE),
    transforms.RandomHorizontalFlip(),
]
random_transforms = transforms.Compose(random_transforms)

下面是初始化新数据集的代码。由于我们已经 `preprocessed_clip` ，我们将使用 `__init__` 函数将其预加载到我们的 GPU 上。我们保留了“即时” CLIP 编码作为示例。它会产生稍好一些的结果，但速度要慢得多。

In [ ]:
class MyDataset(Dataset):
    def __init__(self, csv_path, preprocessed_clip=True):
        self.imgs = []
        self.preprocessed_clip = preprocessed_clip
        if preprocessed_clip:
            self.labels = torch.empty(
                len(data_paths), CLIP_FEATURES, dtype=torch.float, device=device
            )
        
        with open(csv_path, newline='') as csvfile:
            reader = csv.reader(csvfile, delimiter=',')
            for idx, row in enumerate(reader):
                img = Image.open(row[0])
                self.imgs.append(pre_transforms(img).to(device))
                if preprocessed_clip:
                    label = [float(x) for x in row[1:]]
                    self.labels[idx, :] = torch.FloatTensor(label).to(device)

    def __getitem__(self, idx):
        img = random_transforms(self.imgs[idx])
        if self.preprocessed_clip:
            label = self.labels[idx]
        else:
            batch_img = img[None, :, :, :]
            encoded_imgs = clip_model.encode_image(clip_preprocess(batch_img))
            label = encoded_imgs.to(device).float()[0]
        return img, label

    def __len__(self):
        return len(self.imgs)

In [ ]:
train_data = MyDataset(csv_path)
dataloader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)

U-Net 模型的架构与上次相同，但有一个小区别。我们将使用 `CLIP_FEATURES` 的数量，而不是使用类的数量作为 `c_embed_dim` 。之前，`c` 可能代表“类”，但这次，它代表“上下文”。幸运的是，它们都以 `c` 开头，因此我们不需要重构代码来反映这种意图的变化。

In [ ]:
T = 400
B_start = 0.0001
B_end = 0.02
B = torch.linspace(B_start, B_end, T).to(device)

ddpm = ddpm_utils.DDPM(B, device)
model = UNet_utils.UNet(
    T, IMG_CH, IMG_SIZE, down_chs=(256, 256, 512), t_embed_dim=8, c_embed_dim=CLIP_FEATURES
)
print("Num params: ", sum(p.numel() for p in model.parameters()))
model_flowers = torch.compile(model.to(device))

`get_context_mask` 函数会略有变化。由于我们用 CLIP 嵌入替换了分类输入，因此我们不再需要对标签进行独热编码。我们仍会将编码中的值随机设置为 `0` ，以帮助模型在没有上下文的情况下学习。

In [ ]:
def get_context_mask(c, drop_prob):
    c_mask = torch.bernoulli(torch.ones_like(c).float() - drop_prob).to(device)
    return c_mask

我们还重新构建了 `sample_flowers` 函数。这一次，它将以我们的 `text_list` 作为参数并将其转换为 CLIP 编码。`sample_w` 函数基本保持不变，并已移至 [ddpm_utils.py](utils/ddpm_utils.py) 的底部。

In [ ]:
def sample_flowers(text_list):
    text_tokens = clip.tokenize(text_list).to(device)
    c = clip_model.encode_text(text_tokens).float()
    x_gen, x_gen_store = ddpm_utils.sample_w(model, ddpm, INPUT_SIZE, T, c, device)
    return x_gen, x_gen_store

训练时间到了！经过大约 `50` 个 `epochs` 后，模型将开始生成一些可识别的内容，在 `100` 时它将达到最佳状态。觉得如何？生成的图像与您的描述相符吗？

In [ ]:
epochs=100
c_drop_prob = 0.1
lrate = 1e-4
save_dir = "05_images/"

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=lrate)

model.train()
for epoch in range(epochs):
    for step, batch in enumerate(dataloader):
        optimizer.zero_grad()
        t = torch.randint(0, T, (BATCH_SIZE,), device=device).float()
        x, c = batch
        c_mask = get_context_mask(c, c_drop_prob)
        loss = ddpm.get_loss(model_flowers, x, t, c, c_mask)
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch} | Step {step:03d} | Loss: {loss.item()}")
    if epoch % 5 == 0 or epoch == int(epochs - 1):
        x_gen, x_gen_store = sample_flowers(text_list)
        grid = make_grid(x_gen.cpu(), nrow=len(text_list))
        save_image(grid, save_dir + f"image_ep{epoch:02}.png")
        print("saved images in " + save_dir + f" for episode {epoch}")

现在模型已经训练完毕，让我们来玩一玩吧！如果我们给它一个数据集中没有的东西作为提示，会发生什么？或者你能制作出完美的提示来生成你能想象到的图像吗？

制作提示以获得所需结果的艺术称为**提示工程**，正如这里所示，这取决于模型所训练的数据类型。

In [ ]:
# Change me
text_list = [
    "A daisy",
    "A sunflower",
    "A rose"
]

model.eval()
x_gen, x_gen_store = sample_flowers(text_list)
grid = make_grid(x_gen.cpu(), nrow=len(text_list))
other_utils.show_tensor_image([grid])
plt.show()

找到一组您喜欢的图像后，运行下面的单元格将其转换为动画。它将保存到 [05_images/flowers.gif](05_images/flowers.gif)

In [ ]:
grids = [other_utils.to_image(make_grid(x_gen.cpu(), nrow=len(text_list))) for x_gen in x_gen_store]
other_utils.save_animation(grids, "05_images/flowers.gif")

## 5.3 下一步

恭喜您完成了课程！希望您能享受这段旅程，并能创造出值得与亲朋好友分享的东西。

准备好测试您的技能了吗？

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

<center><img src="images/DLI_Header.png" alt="标题" style="width: 400px;"/></center>